In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import imutils
import math
%matplotlib

Using matplotlib backend: TkAgg


In [6]:
def transformation(img, points, angle, row_row):
    cos = math.cos(angle*math.pi/180.0)
    sin = math.sin(angle*math.pi/180.0)
    # definimos el centro normal y el de la transformacion
    (h, w) = img.shape[:2]
    (cx, cy) = (w // 2, h // 2)
    nW = int((w * abs(cos)) + (h * abs(sin)))
    nH = int((w * abs(sin)) + (h * abs(cos)))
    (CX,CY) = (nW // 2, nH // 2)
    #creamos la imagen
    img_T = np.zeros((nH,nW), dtype=np.uint8)
    #tranformamos unicamente los points
    points_T = []
    for p in points:
        x_, y_ = p[0]-cx, p[1]-cy
        X_ = x_*cos-y_*sin
        Y_ = x_*sin+y_*cos
        X,Y = int(round(X_+CX)), int(round(Y_+CY))
        #ya tenemos la pos in T now fill
        img_T[Y][X] = 255
        #almacenamos la posicion en T para saber cual le corresponde del espacio original
        points_T.append((X,Y))
        
    # ya tenemos la imagen con los points en T
    _, contours,_ = cv2.findContours(img_T, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rr_T = []
    for cnt in contours:
        x,y,w_,h_ = cv2.boundingRect(cnt)
        p = (x+w_//2,y+h_//2)
        rr_T.append(p)
    
    # aun puede haber un desorden, ordenamos por el axis x
    i = 0
    ys = []
    while i<len(rr_T):
        pi = rr_T[i]
        i += 1

        count = 0
        while i<len(rr_T) and count < 4: 
            pn = rr_T[i]
            # comprobar si esta en la linea
            if abs(pi[1]-pn[1])> 0.4*row_row: break
            i += 1
            count += 1
        
        ys.append(pi[1])
        
        #ordenamos segun x, para los que pertenecen al mismo row
        idx = rr_T.index(pi)
        rr_T[idx:idx+count+1] = sorted(rr_T[idx:idx+count+1], key=lambda item:item[0], reverse=True)
            
    #compute new row_row
    if len(ys)>1:
        row_row = abs(ys[0]-ys[1])
    
    # ahora calculamos el orden real
    orden = []
    for p in points_T:
        id_T = rr_T.index(p)
        orden.append(id_T)
    
    return orden, img_T, row_row

In [2]:
image = cv2.imread('117.jpg')
output = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_,binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [6]:
circles = cv2.HoughCircles(binary, cv2.HOUGH_GRADIENT, 1, 20, param2=100)

cv2.namedWindow('o',0)
cv2.namedWindow('b',0)
# ensure at least some circles were found
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
 
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
 
    # show the output image
    #cv2.imshow('o', np.hstack([binary, output]))
    cv2.imshow('o',output)
else:
    print'ningun circulo'
cv2.imshow('b',binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

ningun circulo


In [2]:
img1 = cv2.imread('56.jpg')
img2 = cv2.imread('57.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [3]:
parche1 = gray1[232:740,791:1375]
parche2 = gray2[232:740,791:1375]
t,binary1 = cv2.threshold(parche1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
_,binary2 = cv2.threshold(parche2, t, 255, cv2.THRESH_BINARY)

cv2.namedWindow('56',0)
cv2.imshow('56',np.hstack([binary1-binary2, binary1, binary2]))
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('56.jpg',parche1)
cv2.imwrite('57.jpg',parche2)

True

In [42]:
img_rgb = cv2.imread('sample2.jpg')
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('t3.jpg',0)

#_, img_gray = cv2.threshold(img_gray, 168, 255, cv2.THRESH_BINARY)
#_,template = cv2.threshold(template, 168, 255, cv2.THRESH_BINARY)


w, h = template.shape[::-1]

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where( res >= threshold)
#for pt in zip(*loc[::-1]):
 #   cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

# ya tengo el map de valores del template matching

_,res_1 = cv2.threshold(res, 0.8, 1.0, cv2.THRESH_BINARY)
res_1 = np.array(res_1*255, np.uint8)
_, contours,_ = cv2.findContours(res_1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
rr = []
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    #  ahora en la imagen del mapping busquemos los maximos
    _,v,_,p =cv2.minMaxLoc(res[y:y+h,x:x+w])
    p = (p[0]+x,p[1]+y)
    print v, p
    '''
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    suma = [p[0]+p[1] for p in box]
    idx_min = suma.index(min(suma))
    idx_max = suma.index(max(suma))
    x1,y1 = box[idx_min][0], box[idx_min][1]
    x2,y2 = box[idx_max][0], box[idx_max][0]
    #  ahora en la imagen del mapping busquemos los maximos
    _,v,_,p =cv2.minMaxLoc(res[y1:y2,x1:x2])
    p = (p[0]+x1,p[1]+y1)
    print v, p
    '''

cv2.imshow('bin',res_1)
cv2.imshow('tmp',template)
cv2.imshow('res',img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.992649495602 (134, 10)
0.999312162399 (11, 9)


In [65]:
suma = [p[0]+p[1] for p in box]


In [21]:
cv2.minMaxLoc(res)


(-0.1245398074388504, 0.999312162399292, (41, 17), (11, 9))

In [12]:
N = 600
l_best = [0 for i in range(N+5)]
l_first = [0 for i in range(N+5)]

In [7]:
cap = cv2.VideoCapture('calibration_mslifecam.avi')
template = cv2.imread('t6.jpg',0)
area = 2700.5
threshold = 50
row = 240#271
col = 0
flag = True
i_frame = 0
# roi
X_FROM, Y_FROM = 0, 0
X_TO, Y_TO = 0, 0
angle = 0
row_row = 80

cv2.namedWindow('image',0)
cv2.namedWindow('sample',0)
cv2.namedWindow('aux',0)
while(True):
    start = time.time()
    ret, frame = cap.read()
    if ret== False: break
    
    w, h = template.shape[::-1]
    gray_initial = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #si es la primera vez entonces encierra a toda la imagen el ROI
    if i_frame == 0:
        X_FROM, Y_FROM = 0, 0
        X_TO, Y_TO = frame.shape[1]-1, frame.shape[0]-1
    
    gray = gray_initial[Y_FROM:Y_TO , X_FROM: X_TO]
    
    
    
    res = cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
    #_,gray_b = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    #_,template_b = cv2.threshold(template, threshold, 255, cv2.THRESH_BINARY)
    #res = cv2.matchTemplate(gray_b,template_b,cv2.TM_CCOEFF_NORMED)
    
    #for pt in zip(*loc[::-1]):
     #   cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        #template = gray[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
    
    _,res_1 = cv2.threshold(res, 0.5, 1.0, cv2.THRESH_BINARY)
    res_1 = np.array(res_1*255, np.uint8)
    _, contours,_ = cv2.findContours(res_1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rr = []
    values = []
    for cnt in contours:
        x,y,w_,h_ = cv2.boundingRect(cnt)
        #  ahora en la imagen del mapping busquemos los maximos
        _,val,_,p =cv2.minMaxLoc(res[y:y+h_,x:x+w_])
        p = (p[0]+x,p[1]+y)
        rr.append(p)
        values.append(val)
    
    #l_first[i_frame] = len(values)
    #l_best[i_frame] = len(values)
    
    '''
    print len(rr),len(values)
    if i_frame == 2009:
        
        cv2.imshow('image', gray)
        cv2.imshow('aux', template)
        cv2.imshow('sample', res)
        cv2.waitKey(0)
    '''
        
    #actualizar el template
    if len(rr)>0: # comprobar si es que en el frame actual hubo matching
        sort =  sorted(values, reverse=True)#ordeno por el valor de coincidencia
        idxs = [values.index(i) for i in sort]
        #x,y = rr[0]#rr[idxs[0]]
        x,y = rr[idxs[0]]
        pad = int(0.07*max(w,h))
        #TODO:falta comprobar limites
        xi, yi = x-pad, y-pad
        tmp_pad = gray[y-pad:y+h+pad , x-pad:x+w+pad]
        tmp_pad_rgb = frame[y-pad:y+h+pad , x-pad:x+w+pad]
        #print gray.shape, (y-pad, y+h+pad), (x-pad, x+w+pad)
        # comprobar limites del template expandido
        if y-pad<0 or y+h+pad>frame.shape[0] or x-pad<0 or x+w+pad>frame.shape[1]:
            print 'escogiendo otro template'
            assert len(rr)>1, 'no hay mas templates'
            x,y = rr[idxs[1]]
            xi, yi = x-pad, y-pad
            tmp_pad = gray[y-pad:y+h+pad , x-pad:x+w+pad]
            tmp_pad_rgb = frame[y-pad:y+h+pad , x-pad:x+w+pad]
            
        #print frame.shape
        #print y-pad,y+h+pad
        #print x-pad,x+w+pad
        
        # binarizar, find contours, and agrandar
        ret, img_b = cv2.threshold(tmp_pad, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        threshold = ret
        
        _, contours,_ = cv2.findContours(img_b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        cv2.drawContours(tmp_pad_rgb,contours,-1,(0,255,0),1)
        for cnt in contours:
            x_,y_,w_,h_ = cv2.boundingRect(cnt)
            cv2.rectangle(tmp_pad_rgb,(x_,y_),(x_+w_,y_+h_),(255,0,0),1)
        #cv2.imshow('aux', img_b)
        
        # escoger cual contorno pertenece al anillo-por diferencia con el anterior area
        areas_diff = []
        aaa = []
        for cnt in contours:
            #areas_diff.append(abs(cv2.contourArea(cnt) - area))
            #aaa.append(cv2.contourArea(cnt))
            x_,y_,w_,h_ = cv2.boundingRect(cnt)
            areas_diff.append(abs(w_*h_ - area))
            aaa.append(w_*h_)
            
        index = areas_diff.index(min(areas_diff))
        
        #print(area, aaa)
        
        # ahora veamos si este contorno pertecence al anillo
        #new_area = cv2.contourArea(contours[index])
        x_,y_,w_,h_ = cv2.boundingRect(contours[index])
        new_area = w_*h_
        porcentaje = abs(100-new_area*100.0/area)
        #print('%',porcentaje)
        
        if porcentaje < 30.0:
            area = new_area
            cnt = contours[index]
            x,y,w_,h_ = cv2.boundingRect(cnt)
            #print ('w, h',w_,h_)
            pad = int(max(w_,h_)*0.1)
            xf1,yf1 = xi+x-pad, yi+y-pad
            xf2,yf2 = xi+x+w_+pad, yi+y+h_+pad
            #print xf1,yf1
            #print xf2,yf2
            template = gray[yf1:yf2 , xf1:xf2]
        else:
            print 'desboradamiento'

    
    # generar nuevo template para la propagacion
    #encontrar el matching con el menor valor
    rr2 = []
    if len(values)>0:
        idx_v = values.index((min(values)))
        x,y = rr[idx_v]
        w_, h_ = template.shape[::-1]
        template2 = gray[y:y+h_ , x:x+w_]
        
        tmp_pad_rgb2 = frame[y:y+h_, x:x+w_]
         # binarizar, find contours, and agrandar
        ret, img_b2 = cv2.threshold(template2, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        _, contours_,_ = cv2.findContours(img_b2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(tmp_pad_rgb2,contours_,-1,(0,0,255),1)
        
        
        res2 = cv2.matchTemplate(gray,template2,cv2.TM_CCOEFF_NORMED)
        
        _,res2_1 = cv2.threshold(res2, 0.5, 1.0, cv2.THRESH_BINARY)
        res2_1 = np.array(res2_1*255, np.uint8)
        _, contours2,_ = cv2.findContours(res2_1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        rr2 = []
        values2 = []
        for cnt in contours2:
            x,y,w_,h_ = cv2.boundingRect(cnt)
            #  ahora en la imagen del mapping busquemos los maximos
            _,val,_,p =cv2.minMaxLoc(res2[y:y+h_,x:x+w_])
            p = (p[0]+x,p[1]+y)
            rr2.append(p)
    
    
    
    #calculamos el orden de los anillos, TODO comprobar que haya matching
    orden = []
    
    
    if len(rr)>0:
        only_points = np.zeros(res_1.shape,dtype=np.uint8)
        for pt in rr:
            c,r = pt # (x,y)
            only_points[r][c] = 255
        orden, rot,row_row = transformation(only_points, rr,angle, row_row)
    #actualizar el angle para el siguiente frame
    print angle
    if len(rr)>1:
        first = orden.index(0)
        second = orden.index(1)
        delta_y = rr[first][1] - rr[second][1]
        delta_x = rr[first][0] - rr[second][0]
        if delta_x == 0:
            print 'division por cero'
            if abs(angle-90) > abs(angle+90): what_90 = 90
            else: what_90 = -90
            angle_n = what_90*180.0/math.pi
        else:
            m = delta_y*1.0/delta_x
            angle_n = math.atan(m)
            angle_n = (-1.0)*angle_n*180.0/math.pi
            
        if angle_n>(angle-9) and angle_n<(angle+9):
            angle = angle_n
        else:
            print 'desbordamiento de angulo'
        
    #restaurar los matching del template de propagacion
    rr2 = [(t[0]+X_FROM, t[1]+Y_FROM) for t in rr2] 
    
    #actualizacion del roi
    if (len(rr)>0):
        rr = [(t[0]+X_FROM, t[1]+Y_FROM) for t in rr] 

        # encontrar el cuadro ROI
        min_x = min(rr, key=lambda coord: coord[0])
        max_x = max(rr, key=lambda coord: coord[0])
        min_y = min(rr, key=lambda coord: coord[1])
        max_y = max(rr, key=lambda coord: coord[1])
        roi_x1, roi_y1 = min_x[0], min_y[1]
        roi_x2, roi_y2 = max_x[0], max_y[1]
        roi_x2 += w
        roi_y2 += h
        # adicionar un padding
        pad = int(0.1*max(abs(roi_x1-roi_x2),abs(roi_y1-roi_y2)))
        roi_x1 -= pad
        roi_y1 -= pad
        roi_x2 += pad
        roi_y2 += pad
        #comprobar limites
        if roi_x1 <0: roi_x1=0
        if roi_y1 <0: roi_y1=0
        if roi_x2 >(frame.shape[1]-1): roi_x2 = frame.shape[1]-1
        if roi_y2 >(frame.shape[0]-1): roi_y2 = frame.shape[0]-1

        #cv2.rectangle(frame, (roi_x1,roi_y1), (roi_x2, roi_y2), (255,255,0),2)
        # dibujamos el roi actual
        cv2.rectangle(frame, (X_FROM,Y_FROM), (X_TO, Y_TO), (255,255,0),2)

        #actualizamos el nuevo roi para el siguiente frame
        X_FROM, Y_FROM = roi_x1, roi_y1
        X_TO, Y_TO = roi_x2, roi_y2
        
    # dibujar los matching
    for pt in rr2:
        cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2) 
    
    for k,pt in enumerate(rr):
        cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0,255,0), 2)
    for k,pt in enumerate(rr):
        cv2.putText(frame, str(orden[k]), pt, cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 0, 0), 2)
   
    # calcular los centros de cada matching
    rr_centers = []
    for pt in rr:
        img_path = gray[pt[1]:pt[1]+h, pt[0]:pt[0]+w]
        _, img_path_b = cv2.threshold(template2, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        _, contours_,_ = cv2.findContours(img_path_b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        lista_centers = []
        #for c in cnts:
            # compute the center of the contour
        cX, cY = w//2, h//2
        #if len(contours) == 0: cX, cY = w//2, h//2
        #else:
         #   M = cv2.moments(contours[0])
          #  cX = int(M["m10"] / M["m00"])
          #  cY = int(M["m01"] / M["m00"])
        rr_centers.append((cX+pt[0],cY+pt[1]))
    
    # draw the lines, centro de los circulos
    if len(rr_centers)>0:
        orden_sorted =sorted(orden)
        idx = orden.index(orden_sorted[0])
        last_x, last_y = rr_centers[idx] 
        i = 1
        while i<len(rr_centers):
            idx = orden.index(orden_sorted[i])
            new_x, new_y = rr_centers[idx]
            cv2.line(frame,(last_x,last_y),(new_x,new_y),(0,255,255),3)
            last_x, last_y = new_x, new_y
            i += 1
    
    cv2.putText(frame, str(i_frame), (10, 40), cv2.FONT_HERSHEY_SIMPLEX,1.5, (255, 0, 0), 3)
    cv2.putText(frame, str(int(angle)), (10,80), cv2.FONT_HERSHEY_SIMPLEX,1.5, (255, 0, 0), 2)
    
    cv2.imshow('image',frame)
    cv2.imshow('sample',res_1)
    kernel = np.ones((5,5),np.uint8)
    rot = cv2.dilate(rot,kernel,iterations = 2)
    cv2.imshow('aux', rot)
    
    end = time.time()
    print("tiempo es:", end-start)
    
    if i_frame>180000:
        #cv2.imwrite(str(i_frame+2000)+'.jpg', template)
        if cv2.waitKey(0) == 27:
            break
    else:
        if cv2.waitKey(0) == 27:
            break
    
    i_frame += 1
    print ('iteration',i_frame)

cap.release()
cv2.destroyAllWindows()

0
('tiempo es:', 0.11473584175109863)
('iteration', 1)
-2.45403167453
('tiempo es:', 0.03325295448303223)
('iteration', 2)
-2.41950921666
('tiempo es:', 0.021341800689697266)
('iteration', 3)
-2.41950921666
('tiempo es:', 0.03006601333618164)
('iteration', 4)
-3.27048792318
('tiempo es:', 0.020427942276000977)
('iteration', 5)
-3.22452260652
('tiempo es:', 0.0238339900970459)
('iteration', 6)
-4.02826366649
('tiempo es:', 0.01829385757446289)
('iteration', 7)
-4.83041995829
('tiempo es:', 0.023120880126953125)
('iteration', 8)
-5.55297103304
('tiempo es:', 0.021294116973876953)
('iteration', 9)
-6.34019174591
('tiempo es:', 0.02696704864501953)
('iteration', 10)
-6.25403274392
('tiempo es:', 0.020652055740356445)
('iteration', 11)
-7.02839623895
('tiempo es:', 0.020761966705322266)
('iteration', 12)
-7.02839623895
('tiempo es:', 0.020689964294433594)
('iteration', 13)
-7.02839623895
('tiempo es:', 0.028094053268432617)
('iteration', 14)
-5.40379136025
('tiempo es:', 0.02070093154907226

5.78239232236
('tiempo es:', 0.02501988410949707)
('iteration', 121)
5.78239232236
('tiempo es:', 0.021646976470947266)
('iteration', 122)
5.85601358543
('tiempo es:', 0.02532505989074707)
('iteration', 123)
5.85601358543
('tiempo es:', 0.02293109893798828)
('iteration', 124)
5.78239232236
('tiempo es:', 0.03267097473144531)
('iteration', 125)
6.49934627965
('tiempo es:', 0.022111892700195312)
('iteration', 126)
6.49934627965
('tiempo es:', 0.02134394645690918)
('iteration', 127)
5.7105931375
('tiempo es:', 0.021604061126708984)
('iteration', 128)
6.41878673024
('tiempo es:', 0.03450512886047363)
('iteration', 129)
5.7105931375
('tiempo es:', 0.027723073959350586)
('iteration', 130)
6.41878673024
('tiempo es:', 0.02285480499267578)
('iteration', 131)
7.1250163489
('tiempo es:', 0.036150217056274414)
('iteration', 132)
7.82907651006
('tiempo es:', 0.03911590576171875)
('iteration', 133)
8.42696902148
('tiempo es:', 0.029825210571289062)
('iteration', 134)
8.42696902148
('tiempo es:', 0.

('iteration', 235)
1.76239102366
('tiempo es:', 0.02919316291809082)
('iteration', 236)
0.881403996582
('tiempo es:', 0.026720046997070312)
('iteration', 237)
0.868051449746
('tiempo es:', 0.02219700813293457)
('iteration', 238)
0.881403996582
('tiempo es:', 0.02062392234802246)
('iteration', 239)
1.73570458893
('tiempo es:', 0.020639896392822266)
('iteration', 240)
2.6025622025
('tiempo es:', 0.018970012664794922)
('iteration', 241)
2.68377515947
('tiempo es:', 0.02071213722229004)
('iteration', 242)
4.61064931866
('tiempo es:', 0.028828144073486328)
('iteration', 243)
5.7105931375
('tiempo es:', 0.024598121643066406)
('iteration', 244)
5.80672690553
('tiempo es:', 0.0212709903717041)
('iteration', 245)
6.88172363064
('tiempo es:', 0.01864910125732422)
('iteration', 246)
6.88172363064
('tiempo es:', 0.017524003982543945)
('iteration', 247)
6.76617482255
('tiempo es:', 0.018578052520751953)
('iteration', 248)
7.47155917659
('tiempo es:', 0.030405044555664062)
('iteration', 249)
7.35237

In [1]:
x = [i for i in range(N+5)]
plt.plot(x,l_first,'r--')
plt.plot(x,l_best,'b--')

NameError: name 'N' is not defined

In [18]:
x1,y1 = 2,3
x2,y2 = 7,8
angle = math.atan((5.0)/(0.1))*180/math.pi
print angle

88.8542371618


In [6]:
int(round(2.5))

3

In [32]:
def transformation(img, points, angle):
    cos = math.cos(angle*math.pi/180.0)
    sin = math.sin(angle*math.pi/180.0)
    # definimos el centro normal y el de la transformacion
    (h, w) = img.shape[:2]
    (cx, cy) = (w // 2, h // 2)
    nW = int((w * abs(cos)) + (h * abs(sin)))
    nH = int((w * abs(sin)) + (h * abs(cos)))
    (CX,CY) = (nW // 2, nH // 2)
    #creamos la imagen
    img_T = np.zeros((nH,nW), dtype=np.uint8)
    #tranformamos unicamente los points
    points_T = []
    for p in points:
        x_, y_ = p[0]-cx, p[1]-cy
        X_ = x_*cos-y_*sin
        Y_ = x_*sin+y_*cos
        X,Y = int(X_+CX), int(Y_+CY)
        #ya tenemos la pos in T now fill
        img_T[Y][X] = 255
        #almacenamos la posicion en T para saber cual le corresponde del espacio original
        points_T.append((X,Y))
        
    # ya tenemos la imagen con los points en T
    _, contours,_ = cv2.findContours(img_T, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rr_T = []
    for cnt in contours:
        x,y,w_,h_ = cv2.boundingRect(cnt)
        p = (x+w_//2,y+h_//2)
        rr_T.append(p)
        
    # ahora calculamos el orden real
    orden = []
    for p in points_T:
        id_T = rr_T.index(p)
        orden.append(id_T)
    
    return orden, img_T
    
    
img = np.zeros((10,20),dtype=np.uint8)
img[9][1] = 255
img[5][10] = 255
img[5][1] = 255
img[1][18] = 255
img[1][6] = 255

points = [(1,9),(10,5),(1,5),(18,1),(6,1)]

rotate = imutils.rotate_bound(img, 30)
#rotate2 = imutils.rotate_bound(rotate, -20)

orden, T = transformation(img, points,30)
print points
print orden


cv2.namedWindow('ori',0)
cv2.namedWindow('rot',0)
cv2.namedWindow('rot2',0)

cv2.imshow('ori', img)
cv2.imshow('rot',T)
cv2.imshow('rot2',rotate)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(1, 9), (10, 5), (1, 5), (18, 1), (6, 1)]
[2, 1, 3, 0, 4]


In [23]:
a = [(1,2),(8,9)]
a.index((1,3))


ValueError: (1, 3) is not in list

In [4]:
M = cv2.getRotationMatrix2D((10, 5), 20, 1.0)
M

array([[ 0.93969262,  0.34202014, -1.10702692],
       [-0.34202014,  0.93969262,  3.72173833]])

In [37]:
img = cv2.imread('26.jpg')
template = cv2.imread('tx.jpg',0)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
res = cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)

_,res_1 = cv2.threshold(res, 0.5, 1.0, cv2.THRESH_BINARY)
res_1 = np.array(res_1*255, np.uint8)
_, contours,_ = cv2.findContours(res_1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
rr = []
values = []
for cnt in contours:
    x,y,w_,h_ = cv2.boundingRect(cnt)
    #  ahora en la imagen del mapping busquemos los maximos
    _,val,_,p =cv2.minMaxLoc(res[y:y+h_,x:x+w_])
    p = (p[0]+x,p[1]+y)
    rr.append(p)
    values.append(val)

only_points = np.zeros(res_1.shape,dtype=np.uint8)
for pt in rr:
    c,r = pt # (x,y)
    only_points[r][c] = 255

orden, rot = transformation(only_points, rr,10)
print rr
print orden


'''
#calculo el desfase que hay en T y invT
aux2 = np.zeros(res_1.shape, dtype=np.uint8)
cy,cx = int(aux2.shape[0]/2), int(aux2.shape[1]/2)
print cx,cy
aux2[cy][cx] = 255
rot = imutils.rotate_bound(aux2,45)
irot = imutils.rotate_bound(rot,-45)
_,_,_,maximo = cv2.minMaxLoc(irot)
dx, dy = maximo[0]-cx, maximo[1]-cy
print dx, dy

#pasamos al espacio T
rotate = imutils.rotate_bound(res_1, 10)
_, contours,_ = cv2.findContours(res_1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
rr_T = []
for cnt in contours:
    x,y,w_,h_ = cv2.boundingRect(cnt)
    p = (x,y)
    rr_T.append(p)

print rr
print rr_T
'''

cv2.namedWindow('ori',0)
cv2.namedWindow('aux',0)
cv2.namedWindow('rot',0)
cv2.imshow('ori', gray)
cv2.imshow('aux', aux)
cv2.imshow('rot',rot)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(505, 373), (584, 360), (660, 347), (735, 335), (808, 322), (492, 293), (571, 281), (647, 269), (722, 258), (794, 246), (480, 214), (558, 203), (635, 192), (709, 182), (781, 172), (467, 135), (546, 125), (622, 115), (696, 106), (768, 97)]
[4, 3, 2, 1, 0, 9, 8, 7, 6, 5, 14, 13, 12, 11, 10, 19, 18, 17, 16, 15]


In [3]:
tmp = cv2.imread('t3.jpg')
tmp_gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
ret, img_b = cv2.threshold(tmp_gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
_, contours,_ = cv2.findContours(img_b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
suma = [p[0]+p[1] for p in box]
idx_min = suma.index(min(suma))
idx_max = suma.index(max(suma))
x1,y1 = box[idx_min][0], box[idx_min][1]
x2,y2 = box[idx_max][0], box[idx_max][0]
#  ahora en la imagen del mapping busquemos los maximos
#_,v,_,p =cv2.minMaxLoc(res[y1:y2,x1:x2])
dx,dy = abs(x1-x2), abs(y1-y2)
if dx>dy: pad = int(0.07*dx)
else: pad = int(0.07*dy)
x1 -= pad
y1 -= pad
x2 += pad
y2 += pad
print x1,y1
print x2,y2


cv2.drawContours(tmp,contours,-1,(0,0,255),1)

cv2.imshow('img',tmp)
cv2.imshow('asa',img_b)
cv2.waitKey(0)
cv2.destroyAllWindows()

2 2
92 92


In [3]:
tmp_pad_rgb = cv2.imread('t3.jpg')
tmp_pad = cv2.cvtColor(tmp_pad_rgb, cv2.COLOR_BGR2GRAY)
ret, img_b = cv2.threshold(tmp_pad, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
_, contours,_ = cv2.findContours(img_b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
#cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(tmp_pad_rgb,(x,y),(x+w,y+h),(0,255,0),2)
    
    area = cv2.contourArea(cnt)
    print area
    '''
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    suma = [p[0]+p[1] for p in box]
    idx_min = suma.index(min(suma))
    idx_max = suma.index(max(suma))
    x1,y1 = box[idx_min][0], box[idx_min][1]
    x2,y2 = box[idx_max][0], box[idx_max][0]

    print x1, y1
    print x2,y2
    '''

cv2.imshow('asa',tmp_pad_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

5203.5
1411.0


In [8]:
l=[5,8,9]
min(l)

5

In [9]:
print 8*8
assert 7>8, 'no cumple'
print 5*5

64


AssertionError: no cumple

In [38]:
a = [2.5, 3.5, 1.5, 3.2]
sort =  sorted(a, reverse=True)#ordeno por el valor de coincidencia
idxs = [a.index(i) for i in sort]

In [39]:
idxs

[1, 3, 0, 2]

In [37]:
a

[2.5, 3.5, 1.5]

In [25]:
type(b)

NoneType

In [12]:
gray = cv2.imread('2010.jpg',0)
template = cv2.imread('tx.jpg',0)
res = cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
_,res_1 = cv2.threshold(res, 0.6, 1.0, cv2.THRESH_BINARY)
res_1 = np.array(res_1*255, np.uint8)

cv2.namedWindow('image',0)
cv2.namedWindow('sample',0)
cv2.namedWindow('aux',0)

cv2.imshow('image', gray)
cv2.imshow('sample', res_1)
cv2.imshow('aux', res)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
res = cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
_,res_1 = cv2.threshold(res, 0.8, 1.0, cv2.THRESH_BINARY)
cv2.namedWindow('image',0)
cv2.namedWindow('sample',0)
cv2.namedWindow('aux',0)

cv2.imshow('image', gray)
cv2.imshow('sample', template)
cv2.imshow('aux', res_1)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
a = [(6,2),(9,1),(3,3),(10,50),(30,30),(20,20)]
a[0:3] = sorted(a[0:3],key=lambda item:item[1],reverse=True)


In [11]:
rr_T = [(360, 281), (287, 281), (139, 281), (65, 281), (213, 280), (359, 209), (286, 209), (213, 209), (140, 209), (67, 209), (357, 140), (285, 140), (141, 139), (70, 139), (214, 138), (356, 71), (285, 71), (214, 71), (73, 71), (143, 70)]
i = 0
while i<len(rr_T):
    pi = rr_T[i]
    i += 1

    count = 0
    while i<len(rr_T) and count < 4: 
        pn = rr_T[i]
        # comprobar si esta en la linea
        if abs(pi[1]-pn[1])> 10: break
        i += 1
        count += 1
    #print count
    #ordenamos segun x, para los que pertenecen al mismo row
    idx = rr_T.index(pi)
    rr_T[idx:idx+count+1] = sorted(rr_T[idx:idx+count+1], key=lambda item:item[0], reverse=True)


In [8]:
image = cv2.imread('0.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, img_b = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

_, contours,_ = cv2.findContours(img_b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image,contours,-1,(0,255,0),1)
cnts = []
for c in contours:
    area = cv2.contourArea(cnt)
    if area>500 and area<6000:
        cnts.append(c)

cv2.drawContours(image,cnts,-1,(0,255,0),1)
cv2.namedWindow('image', 0)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
isFound

False